In [1]:
from pathlib import Path
from sqlalchemy import create_engine

import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

# Se define la ruta para la base de datos local SQLite
project_name = Path.cwd().parent.parent.name
data_projetc_path = Path.home().joinpath(f'Dropbox/La Casa Del Carpintero/Departamento de Programación/data_projects_git/data_{project_name}')
db_file = 'comisiones.db'
db_file_path_str = str(data_projetc_path.joinpath(db_file))

# Se crea el objeto engine para trabajarlo con los metodos de pandas
engine = create_engine(f'sqlite:///{db_file_path_str}')

In [2]:
file01 = 'clientes_regalos'
file01_path_str = str(data_projetc_path.joinpath(f'comisiones/{file01}.xlsx'))
clientes_regalos = pd.read_excel(file01_path_str, dtype_backend='numpy_nullable')

file02 = 'refacturaciones'
file02_path_str = str(data_projetc_path.joinpath(f'costo_ventas/{file02}.xlsx'))
refacturaciones = pd.read_excel(file02_path_str, dtype_backend='numpy_nullable')

In [3]:
with engine.connect() as conn, conn.begin():

    cobranza_enero = pd.read_sql_table('cobranza_1_2024', conn, dtype_backend='numpy_nullable')
    cobranza_febrero = pd.read_sql_table('cobranza_2_2024', conn, dtype_backend='numpy_nullable')
    cobranza_marzo = pd.read_sql_table('cobranza_3_2024', conn, dtype_backend='numpy_nullable')
    cobranza_abril = pd.read_sql_table('cobranza_4_2024', conn, dtype_backend='numpy_nullable')
    cobranza_mayo = pd.read_sql_table('cobranza_5_2024', conn, dtype_backend='numpy_nullable')
    cobranza_junio = pd.read_sql_table('cobranza_6_2024', conn, dtype_backend='numpy_nullable')
    cobranza_julio = pd.read_sql_table('cobranza_7_2024', conn, dtype_backend='numpy_nullable')
    cobranza_año = pd.read_sql_table('cobranza_año_2024', conn, dtype_backend='numpy_nullable')

    costo_ventas_enero = pd.read_sql_table('costo_ventas_1_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_febrero = pd.read_sql_table('costo_ventas_2_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_marzo = pd.read_sql_table('costo_ventas_3_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_abril = pd.read_sql_table('costo_ventas_4_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_mayo = pd.read_sql_table('costo_ventas_5_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_junio = pd.read_sql_table('costo_ventas_6_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_julio = pd.read_sql_table('costo_ventas_7_2024', conn, dtype_backend='numpy_nullable')
    
    datos_com_2022 = pd.read_sql_table('datos_com_2022', conn, dtype_backend='numpy_nullable')
    datos_com_2023 = pd.read_sql_table('datos_com_2023', conn, dtype_backend='numpy_nullable')
    bonos_limites_comisiones = pd.read_sql_table('bonos_limites_comisiones', conn, dtype_backend='numpy_nullable')

engine.dispose()

costo_ventas_list = [costo_ventas_enero, costo_ventas_febrero, costo_ventas_marzo, costo_ventas_abril, costo_ventas_mayo, costo_ventas_junio, costo_ventas_julio]
costo_ventas_año = pd.concat(costo_ventas_list)

In [4]:
def comisiones_mes_2024_func(mes:int, costo_ventas_list:list) -> pd.DataFrame:

    df = costo_ventas_list[mes-1]
    mes_df = df['invoice_date'].dt.month.unique()[0]

    df['mes'] = mes_df

    gerentes_list = ['Carlos Manuel Guzmán Rios', 'Mariana Araceli Carvajal Flores', 'Rosario Martinez Zarate']

    comisiones_2024_vend = (df.loc[
        ~df['salesperson_name'].isin(gerentes_list)
        ,
                [
                    'salesperson_id',
                    'salesperson_name',
                    'business_model',
                    'mes',
                    'price_subtotal',
                    'utilidad_partida_$_after_cc'
                ]
            ].groupby('salesperson_id')
            .agg({
                'salesperson_name':['first'], 
                'business_model':['first'], 
                'mes':['first'], 
                'price_subtotal':['sum'],
                'utilidad_partida_$_after_cc':['sum']
                })
            .set_axis(
                [
                    'salesperson_name',
                    'business_model',
                    'mes',
                    'facturacion',
                    'utilidad'
                ], axis='columns')
            .reset_index()
    )


    comisiones_2024_ger = pd.DataFrame(
                [
                    [206, 'Mariana Araceli Carvajal Flores','Piso', mes_df],
                    [208, 'Carlos Manuel Guzmán Rios','CE', mes_df],
                    [209, 'Rosario Martinez Zarate', 'Piso', mes_df],
                ],
                columns = 
                    [
                        'salesperson_id',
                        'salesperson_name',
                        'business_model',
                        'mes'
                    ]
            )

    comisiones_2024_ger_values = (df.loc[:,
                [
                    'sale_team_description',
                    'mes',
                    'price_subtotal',
                    'utilidad_partida_$_after_cc'
                ]
            ].groupby('sale_team_description')
            .agg({
                'mes':['first'], 
                'price_subtotal':['sum'],
                'utilidad_partida_$_after_cc':['sum']
                })
            .set_axis(
                [
                    'mes',
                    'facturacion',
                    'utilidad'
                ], axis='columns')
            .reset_index()
    )


    comisiones_2024_ger.loc[
            comisiones_2024_ger['salesperson_name'] == 'Carlos Manuel Guzmán Rios'
            , 
            ['facturacion', 'utilidad']
        ] = (comisiones_2024_ger_values[
                    comisiones_2024_ger_values['sale_team_description'].isin(['A1 CE', 'A2 CE'])
                ]
                .groupby('mes')
                .sum()
                [['facturacion', 'utilidad']]
                .values
            )

    comisiones_2024_ger.loc[
            comisiones_2024_ger['salesperson_name'] == 'Mariana Araceli Carvajal Flores'
            , 
            ['facturacion', 'utilidad']
        ] = comisiones_2024_ger_values[
                    comisiones_2024_ger_values['sale_team_description'] == 'A1 Piso'
                ][['facturacion', 'utilidad']].values

    comisiones_2024_ger.loc[
            comisiones_2024_ger['salesperson_name'] == 'Rosario Martinez Zarate'
            , 
            ['facturacion', 'utilidad']
        ] = comisiones_2024_ger_values[
                    comisiones_2024_ger_values['sale_team_description'] == 'A2 Piso'
                ][['facturacion', 'utilidad']].values


    comisiones_2024 = pd.concat([comisiones_2024_ger, comisiones_2024_vend], ignore_index=True)


    for i in range(len(comisiones_2024)):

        if comisiones_2024.iloc[i]['salesperson_name'] in gerentes_list:
            
            bonos_limites_comisiones_line = bonos_limites_comisiones.loc[
                (bonos_limites_comisiones['business_model'] == comisiones_2024.iloc[i]['business_model'])
                & (bonos_limites_comisiones['is_gerente'] == True)
                & (bonos_limites_comisiones['lim_inf'] <= comisiones_2024.iloc[i]['utilidad'])
                & (bonos_limites_comisiones['lim_sup'] >= comisiones_2024.iloc[i]['utilidad'])
            ]

        else:
            
            bonos_limites_comisiones_line = bonos_limites_comisiones.loc[
                (bonos_limites_comisiones['business_model'] == comisiones_2024.iloc[i]['business_model'])
                & (bonos_limites_comisiones['is_gerente'] == False)
                & (bonos_limites_comisiones['lim_inf'] <= comisiones_2024.iloc[i]['utilidad'])
                & (bonos_limites_comisiones['lim_sup'] >= comisiones_2024.iloc[i]['utilidad'])
            ]

        comisiones_2024.loc[i, ['comision_%']] = bonos_limites_comisiones_line[['%']].iloc[0].values
        comisiones_2024.loc[i, ['comision_$']] = comisiones_2024.loc[i, ['comision_%']].iloc[0] * comisiones_2024.loc[i, ['utilidad']].iloc[0]
        comisiones_2024.loc[i, ['bono_venta_$']] = bonos_limites_comisiones_line[['bono']].iloc[0].values


    return comisiones_2024

In [5]:
comisiones_1_2024 = comisiones_mes_2024_func(1, costo_ventas_list)
comisiones_2_2024 = comisiones_mes_2024_func(2, costo_ventas_list)
comisiones_3_2024 = comisiones_mes_2024_func(3, costo_ventas_list)
comisiones_4_2024 = comisiones_mes_2024_func(4, costo_ventas_list)
comisiones_5_2024 = comisiones_mes_2024_func(5, costo_ventas_list)
comisiones_6_2024 = comisiones_mes_2024_func(6, costo_ventas_list)
comisiones_7_2024 = comisiones_mes_2024_func(7, costo_ventas_list)

comisiones_2024 = pd.concat([
                        comisiones_1_2024,
                        comisiones_2_2024,
                        comisiones_3_2024,
                        comisiones_4_2024,
                        comisiones_5_2024,
                        comisiones_6_2024,
                        comisiones_7_2024
                    ])

In [ ]:
with engine.connect() as conn, conn.begin():  
    comisiones_2024.to_sql(name = 'comisiones_2024', index= False, con = conn, if_exists = 'replace')

engine.dispose()

print('Todas las tablas se ingresaron con éxito')

In [35]:
cobranza = cobranza_año.copy()

cobranza.loc[:, ['fact_año']] = cobranza['fact_doc_date'].dt.year
cobranza.loc[:, ['fact_mes']] = cobranza['fact_doc_date'].dt.month
cobranza.loc[:, ['cobranza_mes']] = cobranza['pay_date'].dt.month

cobranza_paga = cobranza.loc[
                            cobranza['paga_comision'] == True,
                            [
                                'salesperson_id',
                                'salesperson_name',
                                'cobranza_mes',
                                'pay_amount',
                                'fact_año',
                                'fact_mes'
                            ]
                        ]


cobranza_paga.groupby(['salesperson_id', 'salesperson_name', 'cobranza_mes', 'fact_año', 'fact_mes']).sum()

pay_amount
salesperson_id salesperson_name               cobranza_mes fact_año fact_mes            
205            Brenda Luz Acosta Lopez        1            2023     11        362,529.11
                                                                    12        509,519.41
                                                           2024     1         159,834.85
                                                                    2           2,166.66
                                              2            2023     12         46,522.41
...                                                                                  ...
246            Jazmin Rosas Damián            6            2024     7             589.39
                                              7            2024     7         221,616.72
248            Itzel Viridiana Martínez Muñoz 7            2024     7         113,750.33
249            Dayana Guadalupe Salazar Luna  7            2024     7          78,424.61
250            Angye Nathaly Castillo Luna    7            2024     7          81,592.02

[308 rows x 1 columns]

In [11]:
datos_com_2023

,salesperson_id,salesperson_name,V1,C1,V2,C2,V3,C3,V4,C4,...,V8,C8,V9,C9,V10,C10,V11,C11,V12,C12
0,205,Brenda Luz Acosta Lopez,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,"1,873,892.19","21,510.03","1,200,567.44","12,336.75","756,619.80","8,962.43"
1,218,Gladiz Melisa Galvez Espinoza,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,"445,247.78","3,875.34","346,733.12","3,283.35","540,410.57","4,859.61"
2,210,Irma Carvajal Flores,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,"1,914,303.18","20,280.24","1,561,072.02","16,723.57","845,081.98","10,101.28"
3,221,Karla Jaqueline Rivera Hernández,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,"655,857.30","6,228.97","962,898.57","8,043.62","670,175.57","5,349.91"
4,224,Leticia Terán Salinas,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,"128,144.75",565.38,"248,271.55","1,178.20","202,762.89",922.10
5,206,Mariana Araceli Carvajal Flores,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,"1,430,488.40","6,503.48","1,603,457.64","7,086.81","1,529,760.13","6,942.88"
6,211,Mayra Angelica Parada Manjarrez,"1,384,876.49","17,429.15","1,555,187.26","20,762.97","927,142.93","9,703.14","1,050,927.38","13,552.45",...,"1,477,080.95","20,555.57",1304476,"16,487.77","1,757,899.89","24,683.01","1,412,102.93","17,744.85","1,114,001.03","16,281.31"
7,216,Nadia Santos Nava,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,"589,047.02","5,777.15","607,513.73","4,916.97","450,516.61","4,144.39"
8,227,Patricia Flores Pantaleón,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,"5,224.64",<NA>,"231,786.79","2,526.84","229,339.68","2,583.58"
9,212,Reyna Arriaga Flores,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,"600,898.02","5,678.15","735,239.37","5,314.66","437,679.98","3,811.39"


In [12]:
wep = cobranza_año.loc[
            (cobranza_año['paga_comision'] == True)
            & (cobranza_año['salesperson_id'] != 211)
            , 
            ['fact_doc_name', 'fact_doc_date', 'pay_date', 'dias_pago']
        ]

wep['año'] = wep['fact_doc_date'].dt.year
wep['mes'] = wep['fact_doc_date'].dt.month

wep.groupby(['año', 'mes']).count()

fact_doc_name  fact_doc_date  pay_date  dias_pago
año  mes                                                   
2023 10               6              6         6          6
     11              62             62        62         62
     12             379            379       379        379
2024 1             3860           3860      3860       3860
     2             3937           3937      3937       3937
     3             3923           3923      3923       3923
     4             4424           4424      4424       4424
     5             4370           4370      4370       4370
     6             3886           3886      3886       3886
     7             3672           3672      3672       3672